# Setup

In [ ]:
# Imporatamos librerias
import shutil
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
import numpy as np
import glob
import os
import re
from datetime import datetime
from pathlib import Path

In [ ]:
# Instalamos la librería de ultralytics que contiene YOLOv11
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 51.8 MB/s eta 0:00:00


In [ ]:
# Creamos directorio de datasets y importamos ficheros de configuracion
!mkdir datasets
!cp drive/MyDrive/UOC/DELT/DELTv11.yaml .
!cp drive/MyDrive/UOC/DELT/.comet.config .

In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
def full_name_func(source_folder,dataset,file_exclusion_patterns = [],subdirectories_included = [],frames = 4):

    full_name_list = []


    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    if not file.startswith(tuple(file_exclusion_patterns)):
                        src_file = os.path.join(source_images_path, file)
                        subfolder = os.path.basename(Path(src_file).parents[1])
                        if ((subfolder in subdirectories_included) or (not subdirectories_included)):
                            full_name = dataset + '-' + subfolder +'-'+ file
                            full_name_list.append(full_name[:-4])

    full_name_list.sort()

    final_list = [file for i,file in enumerate(full_name_list) if i%frames == 0]


    return final_list

In [ ]:
def drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list):

    os.makedirs(destination_images, exist_ok=True)
    os.makedirs(destination_labels, exist_ok=True)


    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    src_file = os.path.join(source_images_path, file)
                    dst_file = os.path.join(destination_images, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_images, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)
            elif directory == 'labels':
                source_labels_path = os.path.join(root, directory)
                for file in os.listdir(source_labels_path):
                    src_file = os.path.join(source_labels_path, file)
                    dst_file = os.path.join(destination_labels, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_labels, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)

In [ ]:
subs = []
for i in range(102,126):
  subs.append(str(i) + 'FTASK')
# Carpetas que no existen
for item in ['115FTASK','104FTASK','107FTASK','109FTASK','112FTASK',]:
  subs.remove(item)

# Carpetas usadas en train
for item in ['106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']:
  subs.remove(item)

In [ ]:
destination_images = '/content/datasets/DELT/val/images'
destination_labels = '/content/datasets/DELT/val/labels'

source_folder = '/content/drive/MyDrive/UOC/DELT/osaconservation/Arribada2023GT'

dataset = 'osaconservation'

n_frames = 1

s_included = subs
# s_included = []

source_folder = '/content/drive/MyDrive/UOC/DELT/' + dataset
file_list = full_name_func(source_folder,dataset,subdirectories_included = s_included,frames=n_frames)
drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list)


# OSA Conservation + YouTube (talla N)

In [ ]:
model_v = 'osaconservation-youtube-202412061432-yolo11n'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

model = YOLO(weigths_path)

metrics = model.val()

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 112MB/s]
val: Scanning /content/datasets/DELT/val/labels... 1408 images, 848 backgrounds, 0 corrupt: 100%|██████████| 2256/2256 [00:01<00:00, 2047.19it/s]

val: New cache created: /content/datasets/DELT/val/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 141/141 [00:21<00:00,  6.44it/s]


                   all       2256       6973      0.953      0.937      0.978      0.765
Speed: 0.4ms preprocess, 3.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/val


# OSA Conservation + YouTube (talla S)

In [ ]:
model_v = 'osaconservation-youtube-202412061719-yolo11s'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

model = YOLO(weigths_path)

metrics = model.val()

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11s summary (fused): 238 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /content/datasets/DELT/val/labels.cache... 1408 images, 848 backgrounds, 0 corrupt: 100%|██████████| 2256/2256 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 141/141 [00:28<00:00,  5.01it/s]


                   all       2256       6973       0.96      0.941       0.98      0.774
Speed: 0.3ms preprocess, 7.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val2


# OSA Conservation + YouTube (talla M)

In [ ]:
model_v = 'osaconservation-youtube-202412062125-yolo11m'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

model = YOLO(weigths_path)

metrics = model.val()

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


val: Scanning /content/datasets/DELT/val/labels.cache... 1408 images, 848 backgrounds, 0 corrupt: 100%|██████████| 2256/2256 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 141/141 [00:54<00:00,  2.58it/s]


                   all       2256       6973      0.961      0.944       0.98      0.776
Speed: 0.4ms preprocess, 20.2ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val3


# OSA Conservation + YouTube (talla L)

In [ ]:
model_v = 'osaconservation-youtube-202412070005-yolo11l'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

model = YOLO(weigths_path)

metrics = model.val()

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


val: Scanning /content/datasets/DELT/val/labels.cache... 1408 images, 848 backgrounds, 0 corrupt: 100%|██████████| 2256/2256 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 141/141 [01:07<00:00,  2.08it/s]


                   all       2256       6973      0.966      0.942      0.982       0.78
Speed: 0.4ms preprocess, 26.2ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val4


# OSA Conservation + YouTube (talla X)

In [ ]:
model_v = 'osaconservation-youtube-202412070456-yolo11x'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

model = YOLO(weigths_path)

metrics = model.val()

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11x summary (fused): 464 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /content/datasets/DELT/val/labels.cache... 1408 images, 848 backgrounds, 0 corrupt: 100%|██████████| 2256/2256 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 141/141 [02:11<00:00,  1.08it/s]


                   all       2256       6973      0.969      0.946      0.984      0.785
Speed: 0.2ms preprocess, 54.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val5


In [ ]:
from google.colab import runtime
runtime.unassign()